In [1]:
import pandas as pd
%reload_kedro

[05/03/23 16:54:59] INFO     Resolved project path as:                                              ]8;id=919853;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=355812;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\
                             C:\Users\Lenovo\Desktop\work\weather\weather-report.                                  
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/03/23 16:54:59] WARNING  Failed to send data to Heap. Exception of type 'ConnectionError' was     ]8;id=565518;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=751943;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro_telemetry\plugin.py#219\219]8;;\
                             raised.                                                                               

                    INFO     Kedro project weather_report                                           ]8;id=691037;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=158109;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\ipython\__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=652242;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=662952;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\ipython\__init__.py#104\104]8;;\
                             'pipelines'                                                                           

In [2]:
catalog.list()

['weather_aus_raw', 'parameters']

In [3]:
df = catalog.load("weather_aus_raw")

[05/03/23 16:58:53] INFO     Loading data from 'weather_aus_raw' (CSVDataSet)...                ]8;id=692648;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=699413;file://C:\Users\Lenovo\desktop\work\weather\venv\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [4]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [5]:
def extract_training_data(df):
  """Drop date column from dataframe.
  Arg=df
  output=Dataframe with no date column."""
  df=df.drop(["Date"],axis=1)
  df1=df[df['RainTomorrow'].notna()] 
  return df1

In [6]:
def treat_missing(df1):
  """treat missing values with ffill and bfill method

  Arg=df_training_data

  output=nan values from training data get fill with the ffill and bfill method."""
  df1_treat_training_data=df1.fillna(method="ffill",axis=0).fillna(method="bfill",axis=0)
  return df1_treat_training_data

In [7]:
def training_data_split(df1_treat_training_data):
  """Training data spliting ie.separating predictors and response variables.

  Arg=df_treat_training_data

  Output=Target variable get separated from main traing data."""
  X_training=df1_treat_training_data.drop(["RainTomorrow"],axis=1)
  y_training=df1_treat_training_data["RainTomorrow"]
  return X_training, y_training

In [8]:
def lebel_encoding_filling_null(X_training):
  """Lebal encoding on the discrite varibales.

  Arg=X_training

  Output=converting categorical data into numeric variable."""
  
  label_encoder = preprocessing.LabelEncoder()
  
  X_training["Location"]=label_encoder.fit_transform(X_training["Location"])
  X_training["WindGustDir"]=label_encoder.fit_transform(X_training["WindGustDir"])
  X_training["WindDir9am"]=label_encoder.fit_transform(X_training["WindDir9am"])
  X_training["WindDir3pm"]=label_encoder.fit_transform(X_training["WindDir3pm"])
  X_training["RainToday"]=label_encoder.fit_transform(X_training["RainToday"])
  df2 = X_training.copy()
  return df2

In [9]:
def split_data(df2,y_training):
  """Split data into features and targets training and test set.
  Args:
    data;data containing features and targets
  returns:
    split data
 """
  

  X_train,X_test,y_train,y_test=train_test_split(df2,y_training,random_state=0,test_size=0.20)

  return X_train,X_test,y_train,y_test 

In [10]:
def train_model(X_train,X_test,y_train,y_test ):
  """Trains the logistic regression model.
  Args:
      X_train:Training data of independent features
      y_train:Training data for price
  Returns:
      Trined model.
 """
  #instantiate the model
  logreg=LogisticRegression(solver='liblinear',random_state=0)

  #fit the model
  logreg = logreg.fit(X_train,y_train)
  return logreg


In [11]:
def evaluate_model(logreg:LogisticRegression, X_test:pd.DataFrame, y_test:pd.Series):
  """calculate and logs the coefficient of determination.
  Args:
      logreg:Trined model
      X_test:Testing data of independent features
      y_test:Testing data for price
  Returns:
      y_pred_test:prediction on x test
      acc        :accuracy_score
  """
  y_pred_test=logreg.predict(X_test)
  #Check accuracy score
  
  acc=(accuracy_score(y_test,y_pred_test))
  print('model accuracy_score: ',acc)
  return y_pred_test,acc

In [ ]:
def inferance_data(df):
  """Drop date column from dataframe.
  Arg=df
  output=Dataframe with no date column."""
  df=df.drop(["Date"],axis=1)
  df_inferance=df[df['RainTomorrow'].isnana()] 
  return df_inferance

In [ ]:
def treat_missing_inferance(df_inferance):
  """treat missing values with ffill and bfill method

  Arg=df_training_data

  output=nan values from training data get fill with the ffill and bfill method."""
  df1_treat_inferance_data=df_inferance.fillna(method="ffill",axis=0).fillna(method="bfill",axis=0)
  return df1_treat_inferance_data

In [ ]:
def inferance_data_split(df1_treat_inferance_data):
  """Training data spliting ie.separating predictors and response variables.

  Arg=df_treat_training_data

  Output=Target variable get separated from main traing data."""
  X_inferance=df1_treat_inferance_data.drop(["RainTomorrow"],axis=1)
  y_inferance=df1_treat_inferance_data["RainTomorrow"]
  return X_inferance, y_inferance

In [ ]:
def lebel_encoding_filling_null(X_inferance):
  """Lebal encoding on the discrite varibales.

  Arg=X_training

  Output=converting categorical data into numeric variable."""
  
  label_encoder = preprocessing.LabelEncoder()
  
  X_inferance["Location"]=label_encoder.fit_transform(X_inferance["Location"])
  X_inferance["WindGustDir"]=label_encoder.fit_transform(X_inferance["WindGustDir"])
  X_inferance["WindDir9am"]=label_encoder.fit_transform(X_inferance["WindDir9am"])
  X_inferance["WindDir3pm"]=label_encoder.fit_transform(X_inferance["WindDir3pm"])
  X_inferance["RainToday"]=label_encoder.fit_transform(X_inferance["RainToday"])
  df2_inferance = X_inferance.copy()
  return df2_inferance